In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from processingQst import *
from similarityFcts import *
from tqdm import tqdm
import pandas as pd

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


/home/omayya/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-03 20:56:14.171665: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-03 20:56:14.171812: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (omayya-vm): /proc/driver/nvidia/version does not exist
/home/omayya/.local/lib/python3.10/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'fr_core_news_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available up

In [2]:
df = pd.read_csv("./input/csv/dev.csv",delimiter=";")

Qst = df["question"]
id  = df["id"]
nbrRepAll = df["nbr_correct_answers"]
RepListIndxAll = df["correct_answers"]
nbrRep = 5
print(len(Qst))
data_list = []
RepListIndx = 'abcde'

312


In [3]:
corpusMS = openJson("output/corpusMS.json")
corpusMrk = openJson("output/corpusMerck.json")

In [4]:
def getRepDBV2(keywordsQ,nwQstp,corpusMS,corpusMrk):
    keywordsR_MS = [item['keywords'] for item in corpusMS]
    keywordsR_Mrk = [item['keywords'] for item in corpusMrk]

    str_keywordsQ = " ".join(keywordsQ)
    str_keywordsR_MS = [" ".join(lst) for lst in keywordsR_MS]
    str_keywordsR_Mrk = [" ".join(lst) for lst in keywordsR_Mrk]

    vectorizer = CountVectorizer()

    X_Mrk = vectorizer.fit_transform([str_keywordsQ] + str_keywordsR_Mrk)
    v1_Mrk = X_Mrk[0]
    v2_Mrk = X_Mrk[1:]
    similarity_Mrk = cosine_similarity(v1_Mrk, v2_Mrk)[0]
    indx_Mrk = np.argmax(similarity_Mrk)
    print(indx_Mrk)
    _, rate_Mrk = ComMembList(keywordsQ, keywordsR_Mrk[indx_Mrk-1], tol=90)

    print(rate_Mrk)

    # print(f"La liste la plus similaire est la liste {indx_Mrk} : {keywordsR_Mrk[indx_Mrk-1]}")
    
    X_MS = vectorizer.fit_transform([str_keywordsQ] + str_keywordsR_MS)
    v1_MS = X_MS[0]
    v2_MS = X_MS[1:]
    similarity_MS = cosine_similarity(v1_MS, v2_MS)[0]
    indx_MS = np.argmax(similarity_MS)
    print(indx_MS)

    _, rate_MS = ComMembList(keywordsQ, keywordsR_Mrk[indx_MS-1], tol=90)

    print(rate_MS)

    # print(f"La liste la plus similaire est la liste {indx_MS} : {keywordsR_MS[indx_MS-1]}")

    return indx_Mrk

In [5]:
for cpt in range(len(Qst)):
# for cpt in tqdm(range(len(Qst))):
    if (cpt > 0):
        break
    elif (cpt>=0): 
        qst = Qst[cpt]
        nwQst = ''
        listMed = []
        listMedR = []
        keywordsQ = []
        listMedQst = []
        ListMedAns = ''
        Spword = False
        data_dict = {}

        # Détécter la négation 
        if(isRequestWrongAns(qst)):
            WrongQst = True 
            # print(WrongQst)
        else: 
            WrongQst = False 
            # print(WrongQst)
        
        # supprimer les parenthèses 
        qst = del_betParenthese(qst)

        # fragementation des questions selon les ","
        fragQst = splitQst(qst)

        for icpt in range(len(fragQst)):
            if not (fragQst[icpt]==''):
                nwQst,Spword,listMed = recoverMedFrag(fragQst,icpt,nwQst,Spword)
                listMedQst.extend(listMed)
        nwQst = nwQst.lstrip()
        nwQstp = ''
        for j in range(1): # 5 réponses 
            Ans = df[f"answers.{RepListIndx[j]}"][cpt] 
            Ans = Ans.lstrip()
            listMedR = RecoverListMed(Ans)
            keywordsQ.extend(listMedQst)
            keywordsQ.extend(listMedR)
            if Spword:
                nwQstp = replaceAtFront(Ans) + nwQst + '.'
            else:
                nwQstp = nwQst + ' '+ Ans.lower() + '.' 
            # Appliquer la négation 
            if WrongQst == True:
                nwQstp = appliquer_negation(nwQstp.lower())
            
            print(keywordsQ)
            # test de similarité 
            indx = getRepDBV2(keywordsQ,nwQstp,corpusMS,corpusMrk)
            # taux_sim = getRepDB(keywordsQ,nwQstp)
            # print(taux_sim)
            # print(indx)
            # créer un dataframe pour les données actuelles
#             data_dict = {'id' : id[cpt], 'ListMed': keywordsQ, 'qst': nwQstp}
#             data_list.append(data_dict)
#             nwQstp = ''
#             keywordsQ = []

# writeJson("output/Qst_dev.json",data_list)




['adrénaline', 'stimule', 'processus', 'métabolique', 'glycolyse']
15285
0
50
0
